Reto: Mercadotecnia telefónica con aprendizaje supervisado

Act:DS_C6_SC3 Solucion_Reto_SC_63

Eduardo Frias Rosales

Introducción:  
Los programas de telemarketing se han venido usando por las empresas durante años como una forma de comunicarse con clientes potenciales de manera directa por vía telefónica. En los últimos años el uso de las técnicas de inteligencia artificial ha permitido potencializar el impacto de estos programas de mercadotecnia.

Un banco ha lanzado un nuevo plan de inversión a largo plazo a través de un programa de telemarketing y desea saber que tan exitoso se puede considerar este programa utilizando técnicas de aprendizaje supervisado. El programa es lanzado a clientes del banco, por lo que este estudio ayudará a optimizar los esfuerzos al conocer las características de los clientes más propensos a adquirir dicho plan de inversión.

 En la presente actividad tu objetivo es predecir si el entrevistado adquirirá el plan de inversión bancario de acuerdo con varias características que se conocen de dicho cliente.

Objetivo:  

Diseñar e implementar un modelo de aprendizaje supervisado que pueda predecir si una persona adquiere o no un plan de inversión bancaria a través de una entrevista telefónica.

In [30]:
import pandas as pd
import numpy as np
from google.colab import drive

# Carga los datos del archivo CSV
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Aprendizaje supervisado/bank_marketing_RETO_DS_AS.csv')

# Imprime las primeras filas del DataFrame
print(data.head())

# Obtener el número de registros
num_registros = data.shape[0]

# Obtener el total de variables
num_variables = data.shape[1]

# Obtener el tipo de cada variable
tipos_variables = data.dtypes

# Obtener la cantidad de datos perdidos de cada variable
datos_perdidos = data.isnull().sum()

# Imprimir la información
print(f"\nNúmero de registros: {num_registros}")
print(f"Total de variables: {num_variables}")
print("\nTipo de cada variable:")
print(tipos_variables)
print("\nCantidad de datos perdidos por variable:")
print(datos_perdidos)

   age            job  marital  education default  balance housing loan  \
0   31  self-employed  married   tertiary      no     2666      no   no   
1   29     unemployed   single    unknown      no     1584      no   no   
2   41    blue-collar  married  secondary      no     2152     yes   no   
3   50    blue-collar  married  secondary      no       84     yes   no   
4   40         admin.  married  secondary      no        0      no   no   

    contact  day month  duration  campaign  pdays  previous poutcome    y  
0  cellular   10   nov       318         2     97         6  success  yes  
1  cellular    6   sep       245         1     -1         0  unknown  yes  
2  cellular   17   nov       369         1     -1         0  unknown   no  
3  cellular   17   jul        18         8     -1         0  unknown   no  
4  cellular   28   jul       496         2    182        11  success  yes  

Número de registros: 9000
Total de variables: 17

Tipo de cada variable:
age           int64

In [31]:
#6.	Transforma las variables categóricas de manera que puedan ser tratadas numéricamente. Justifica si utilizas LabelEncoder o OneHotEcoder.

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Seleccionar las variables categóricas
categorical_features = data.select_dtypes(include=['object']).columns

# Aplicar LabelEncoder o OneHotEncoder según corresponda
for feature in categorical_features:
  # Si la variable tiene pocas categorías (ej. binaria o con pocas opciones), usar LabelEncoder
  if data[feature].nunique() <= 2:
    le = LabelEncoder()
    data[feature] = le.fit_transform(data[feature])
  # Si la variable tiene muchas categorías, usar OneHotEncoder
  else:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    encoded_data = ohe.fit_transform(data[[feature]])
    encoded_df = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out([feature]))
    data = pd.concat([data, encoded_df], axis=1)
    data = data.drop(feature, axis=1)

print(data.head())

   age  default  balance  housing  loan  day  duration  campaign  pdays  \
0   31        0     2666        0     0   10       318         2     97   
1   29        0     1584        0     0    6       245         1     -1   
2   41        0     2152        1     0   17       369         1     -1   
3   50        0       84        1     0   17        18         8     -1   
4   40        0        0        0     0   28       496         2    182   

   previous  ...  month_jun  month_mar  month_may  month_nov  month_oct  \
0         6  ...        0.0        0.0        0.0        1.0        0.0   
1         0  ...        0.0        0.0        0.0        0.0        0.0   
2         0  ...        0.0        0.0        0.0        1.0        0.0   
3         0  ...        0.0        0.0        0.0        0.0        0.0   
4        11  ...        0.0        0.0        0.0        0.0        0.0   

   month_sep  poutcome_failure  poutcome_other  poutcome_success  \
0        0.0               0.0

In [32]:
#7.	Transforma las variables numéricas en los casos que se tenga algún tipo de sesgo.
# Importar la librería necesaria para la transformación de datos
from sklearn.preprocessing import PowerTransformer

# Identificar variables numéricas con sesgo
# Calcula la asimetría de cada variable numérica:
numeric_features = data.select_dtypes(include=['number']).columns
skewness = data[numeric_features].skew()
print("Asimetría de las variables numéricas:\n", skewness)

# Selecciona las variables con un alto grado de asimetría (skewness > 0.5)
skewed_features = skewness[abs(skewness) > 0.5].index

# Aplica la transformación Box-Cox o Yeo-Johnson a las variables con sesgo
for feature in skewed_features:
  # Si la variable tiene valores negativos, usar Yeo-Johnson
  if data[feature].min() <= 0:
    transformer = PowerTransformer(method='yeo-johnson')
  # Si la variable tiene solo valores positivos, usar Box-Cox
  else:
    transformer = PowerTransformer(method='box-cox')
  data[feature] = transformer.fit_transform(data[[feature]])

print(data.head())

Asimetría de las variables numéricas:
 age                     0.801429
default                 7.981424
balance                 7.280036
housing                 0.028452
loan                    2.186231
day                     0.117315
duration                2.184045
campaign                5.392712
pdays                   2.349177
previous                7.682286
y                       0.320997
job_admin.              2.468144
job_blue-collar         1.601081
job_entrepreneur        5.863734
job_housemaid           5.798679
job_management          1.337302
job_retired             3.496606
job_self-employed       5.025767
job_services            2.961379
job_student             5.088290
job_technician          1.805352
job_unemployed          5.298786
job_unknown            13.173235
marital_divorced        2.407201
marital_married        -0.280085
marital_single          0.795553
education_primary       2.096803
education_secondary     0.006223
education_tertiary      0.764769
educ

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

# 8. Considera la variable “y” como la variable de salida y el resto de las variables como las variables de entrada.
X = data.drop('y', axis=1)
y = data['y']

# 9. Particiona los datos en los conjuntos de entrenamiento, validación y prueba en 60%, 20% y 20%, respectivamente.
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]}")
print(f"Tamaño del conjunto de validación: {X_val.shape[0]}")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]}")

# 10. Aplica el modelo Regresión Logística en el conjunto de entrenamiento.
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Valida el modelo con las predicciones del conjunto de validación.
y_pred = model.predict(X_val)

# Calcula la matriz de confusión.
conf_matrix = confusion_matrix(y_val, y_pred)
accuracy = accuracy_score(y_val, y_pred)

print("\nMatriz de confusión en validación:")
print(conf_matrix)
print(f"\nPrecisión del modelo: {accuracy * 100:.2f}%")


Tamaño del conjunto de entrenamiento: 5400
Tamaño del conjunto de validación: 1800
Tamaño del conjunto de prueba: 1800

Matriz de confusión en validación:
[[904 154]
 [167 575]]

Precisión del modelo: 82.17%


In [41]:
# 11.	Aplica el modelo Red Neuronal en el conjunto de entrenamiento. Valida el modelo con las predicciones del conjunto de validación y su matriz de confusión. Ajusta los parámetros del modelo hasta obtener tu mejor modelo, entre ellos el número de neuronas y capas ocultas.

from sklearn.neural_network import MLPClassifier

# Define el modelo de Red Neuronal
model_nn = MLPClassifier(hidden_layer_sizes=(100, 50),
                         max_iter=500,
                         activation='relu',
                         solver='adam',
                         random_state=42)

# Entrena el modelo con los datos de entrenamiento
model_nn.fit(X_train, y_train)

# Realiza predicciones en el conjunto de validación
y_pred_nn = model_nn.predict(X_val)

# Calcula la matriz de confusión
confusion_mat_nn = confusion_matrix(y_val, y_pred_nn)
accuracy_nn = accuracy_score(y_val, y_pred_nn)

print("Matriz de confusión en validación (Red Neuronal):")
print(confusion_mat_nn)
print(f"\nPrecisión del modelo (Red Neuronal): {accuracy_nn * 100:.2f}%")


Matriz de confusión en validación (Red Neuronal):
[[893 165]
 [189 553]]

Precisión del modelo (Red Neuronal): 80.33%


In [43]:
# Selecciona el mejor modelo (Regresion Logística en este caso con random_state=42)
mejor_modelo = model if accuracy > accuracy_nn else model_nn

# Realiza predicciones en el conjunto de prueba
y_test_pred = mejor_modelo.predict(X_test)

# Matriz de confusión en el conjunto de prueba
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Matriz de confusión en prueba:")
print(conf_matrix_test)
print(f"\nPrecisión final del modelo: {accuracy_test * 100:.2f}%")


Matriz de confusión en prueba:
[[865 140]
 [166 629]]

Precisión final del modelo: 83.00%


Conclusión:
13.	Incluye tus conclusiones del problema, en particular, ¿qué puedes decir acerca del uso de técnicas de inteligencia artificial en problemas de mercadotecnia?

El uso de estos algoritmos como la Regresión Logística y las Redes Neuronales permitió predecir con precisión si un cliente realizará una acción específica, lo que optimiza las estrategias de marketing al segmentar audiencias y focalizar esfuerzos en los clientes con más probabilidad de éxito. Estas técnicas mejoran la toma de decisiones basada en datos, permiten manejar grandes volúmenes de información y ajustarse a las necesidades del negocio.